# CNN

Dans ce notebook nous allons implementer un CNN sur le dataset cats vs Dogs

Nous utliserons tensorboard pour surveiller le comportement du modele pendant son entrainement, par exemple pour detecter si le modele overfit

enfin nous allons remedier au probleme ainsi detecté

Le notebook comporte 3 parties

- dataset, chargement, visualisation et formatage
- definition, compilation du modele
- entrainement, observation et remediation



In [ ]:
# pour charger l'extension tensorboard
%load_ext tensorboard

In [ ]:
# Installer kagglehub pour télécharger le dataset
!pip -q install kagglehub

import kagglehub
import os



In [ ]:

# Download latest version
path = kagglehub.dataset_download("birajsth/cats-and-dogs-filtered")

print("Path to dataset files:", path)


In [ ]:
# kagglehub télécharge et dézippe automatiquement

# Le dataset est dans un sous-dossier "cats_and_dogs_filtered"
base_dir = os.path.join(path, "cats_and_dogs_filtered")
base_dir


# Dataset

- Ou se trouve le dataset ?
- De quoi est il constitué ?


le dataset est scindé en une partie d'entrainement et une partie de test.


Qui a t il dans le  repertoires de train ?


In [ ]:
!ls -al {base_dir}
!ls -al {base_dir}/train/cats | head


## Explorer le dataset

On defini les repertoires

In [ ]:
import os
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )


Les fichiers `train_cat_fnames` et `train_dog_fnames` contiennent les noms des fichiers

In [ ]:
train_cat_fnames[:10]

### structure des données

pour etre commpatible avec les outils de tranformation Keras

```
/data/
    /train/
        /class1/
            img1.jpg
            img2.jpg
            ...
        /class2/
            img1.jpg
            img2.jpg
            ...
    /validation/
        /class1/
            img1.jpg
            img2.jpg
            ...
        /class2/
            img1.jpg
            img2.jpg
            ...

```



# Chats et chiens

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# images a partir de l'index :
pic_index = 250

# list des 8 images de chats et chiens
next_cat_pix = [os.path.join(train_cats_dir, fname) for fname in train_cat_fnames[ pic_index:pic_index +8] ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) for fname in train_dog_fnames[ pic_index:pic_index +8]]


# un grid de 4x4

fig, ax = plt.subplots(ncols*4, nrows*4, figsize = (12,12))
for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


Adorables!

# formatter les images

on va
- normalizer les valeurs (diviser par 255)
- utiliser  [_ImageDataGenerator_](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) pour traiter toutes les images des repertoires d'entrainement et de test

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=16,
                                                    class_mode='binary',
                                                    target_size=(150, 150))
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=16,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


# Simple CNN



### Description du modèle
- **Entrée** : images 150×150×3 (RGB).
- **Extraction de motifs** : une couche Conv2D apprend des filtres locaux (bords, textures).
- **Réduction** : MaxPooling2D diminue la résolution et garde l’essentiel.
- **Aplatissement** : Flatten transforme les cartes de caractéristiques en vecteur.
- **Apprentissage non‑linéaire** : Dense(128) pour combiner les features.
- **Régularisation** : Dropout(0.2) limite le sur‑apprentissage.
- **Sortie binaire** : Dense(1) + sigmoïde → probabilité chat vs chien.


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model():
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 150x150 with 3 bytes color
        tf.keras.layers.Input(shape=(150, 150, 3)),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # <ajouter des layers Conv2D et MaxPooling ?>
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


    model.compile(optimizer=RMSprop(learning_rate=0.01),
                loss='binary_crossentropy',
                metrics = ['accuracy'])
    return model


In [ ]:
model = create_model()
model.summary()

# Monitorer le training avec tensorboard

On crée un callback TensorBoard

In [ ]:
import datetime, os
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#  https://keras.io/api/callbacks/tensorboard/
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Lancer tensorboard

Lancer tensorboard avant le `fit()` permet de voir l'évolution du modèle et de ses performances durant l'entrainement et donc d'arrêter l'entrainement si les choses ne vont pas bien


- Le tableau de bord **Scalars** montre comment la perte et les paramètres changent à chaque époque. Vous pouvez également l'utiliser pour suivre la vitesse d'entraînement, le taux d'apprentissage et d'autres valeurs scalaires.
- Le tableau de bord **Graphs** vous permet de visualiser votre modèle. Dans ce cas, le graphique Keras des couches s'affiche, ce qui peut vous aider à vous assurer qu'il est correctement construit.
- Les tableaux de bord **Histograms** montrent la distribution d'un Tensor au fil du temps. Cela peut être utile pour visualiser les poids et les biais et vérifier qu'ils changent de manière attendue.

In [ ]:
# %tensorboard --logdir logs

fit le modele en rajoutant le callbak vers tensorboard

On entraîne le modèle et on enregistre l'historique d'apprentissage.


In [ ]:
history = model.fit(
            train_generator,
            epochs=5,
            validation_data=validation_generator,
            verbose=2,
            callbacks=[tensorboard_callback]
            )

On évalue les performances finales sur le jeu de validation.


In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

On évalue les performances finales sur le jeu de validation.


In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator, steps=train_generator.samples // train_generator.batch_size)
print(f'Train loss: {train_loss}')
print(f'Train accuracy: {train_accuracy}')

On exécute cette étape du pipeline d'entraînement.
